In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'

train_original = pd.read_pickle(PATH + 'train_0.pkl')
test = pd.read_pickle(PATH + 'test_0.pkl')

'done'

'done'

In [2]:
target = train_original['TARGET']
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train_original.columns if x not in ignore_cols]
train = train_original[features]
test = test[features]

train.shape

(307511, 2036)

In [3]:
%%time

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_params = {
    'num_threads': 16,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 48,
    'num_iteration': 10000,
    'verbose': 100,
    'colsample_bytree': .8,
    'subsample': .9,
    'max_depth': 7,
    'reg_alpha': .1,
    'reg_lambda': .1, 
    'min_split_gain': .01,
    'min_child_weight': 1
}

lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=[],
                          free_raw_data=False)

cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=200,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[200]	cv_agg's auc: 0.754561 + 0.00391674
[400]	cv_agg's auc: 0.769436 + 0.00346706
[600]	cv_agg's auc: 0.780437 + 0.00321731
[800]	cv_agg's auc: 0.785806 + 0.00307016


KeyboardInterrupt: 

In [4]:
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=int(optimum_boost_rounds*1.1))

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_cleaned['SK_ID_CURR'], 'TARGET': y_pred})
print('done')

NameError: name 'optimum_boost_rounds' is not defined

In [ ]:
print((out_df['TARGET']>0.5).sum())

In [ ]:
import matplotlib.pyplot as plt
out_df[['TARGET']].hist()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, (ax, ax1) = plt.subplots(1, 2, figsize=[15, 7])
lgbm.plot_importance(clf, ax=ax, max_num_features=30, importance_type='split')
lgbm.plot_importance(clf, ax=ax1, max_num_features=30, importance_type='gain')
ax.set_title('Importance by splits')
ax1.set_title('Importance by gain')
plt.tight_layout()
plt.show()

In [ ]:
out_df.to_csv(PATH + 'submission/submission_0.csv', index=False)
print('done')

In [ ]:
import json

y = [x for x in zip(clf.feature_name(), clf.feature_importance('gain'))]
y.sort(key=lambda x: x[1], reverse=True)
y = [(i, j) for i, j in enumerate(y)]
y
# with open('./importance_gain.json') as f:
#     json.dump(x, f)

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('split'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x
# with open('./importance_split.json') as f:
#     json.dump(x, f)